In [1]:
# Primitive functions
import torch

def vector(*arg):
    if len(arg) == 0:
        return torch.tensor([])
    # general case
    try:
        return torch.stack(arg, dim=0)
    
    # for concatenation of many vectors
    except RuntimeError:
        dim = len(arg[0].shape) - 1
        return torch.cat(arg, dim=dim)
    
    # for distribution objects
    except TypeError:
        return list(arg)

def get(v, i):
    if type(i) is str:
        return v[i]
    return v[int(i.item())]

def put(v, i, c):
    if type(i) is str:
        v[i] = c
    else:
        v[int(i.item())] = c
    return v

def first(v):
    return v[0]

def second(v):
    return v[1]

def last(v):
    return v[-1]

def append(v, c):
    return torch.cat((v, c.unsqueeze(dim=0)), dim=0)

def hashmap(*v):
    hm = {}
    i = 0
    while i < len(v):
        if type(v[i]) is str:
            hm[v[i]] = v[i+1]
        else:
            hm[v[i].item()] = v[i+1]
        i+=2
    return hm

def less_than(*args):
    return args[0] < args[1]

def rest(v):
    return v[1:]

def l(*arg):
    return list(arg)

def cons(x, l):
    return [x] + l  

def equal(x, y):
    return torch.tensor(x.item() == y.item())

def and_fn(x, y):
    return x and y

def or_fn(x, y):
    return x or y

def dirac(x):
    # approximate with a normal distribution but with very small std
    return torch.distributions.Normal(x, 0.001)

def greater_than(x, y):
    return x > y

def empty(v):
    return len(v) == 0

def peek(v):
    return v[-1]

def push_addr(alpha, value):
    return alpha + value

funcprimitives = {
    "vector": vector,
    "get": get,
    "put": put,
    "first": first,
    "last": last,
    "append": append,
    "hash-map": hashmap,
    "less_than": less_than,
    "second": second,
    "rest": rest,
    "conj": append,
    "list": l,
    "cons": cons,
    "=": equal,
    "and": and_fn,
    "or": or_fn,
    "dirac": dirac,
    ">": greater_than,
    "empty?": empty,
    "peek": peek,
    "push_addr": push_addr,
}

In [2]:
import copy

class Env(dict):
    "An environment: a dict of {'var': val} pairs, with an outer Env."
    def __init__(self, parms=(), args=(), outer=None):
        self.update(zip(parms, args))
        self.outer = outer 
        if outer is not None:
            self.outer = copy.deepcopy(outer)
    def find(self, var):
        "Find the innermost Env where var appears."
        return self if (var in self) else self.outer.find(var)

class Procedure(object):
    "A user-defined Scheme procedure."
    def __init__(self, parms, body, env):
        self.parms, self.body, self.env = parms, body, env
    def __call__(self, *args): 
        return evaluate_helper(self.body, Env(self.parms, args, self.env))

In [12]:
def standard_env() -> Env:
    "An environment with some Scheme standard procedures."
    env = Env()
    env.update({'alpha' : ''}) 
    env.update({'normal': torch.distributions.Normal,
       'sqrt': torch.sqrt,
       '+': torch.add,
       '-': torch.sub,
       '*': torch.mul,
       '/': torch.div,
       'beta': torch.distributions.Beta,
       'gamma': torch.distributions.Gamma,
       'dirichlet': torch.distributions.Dirichlet,
       'exponential': torch.distributions.Exponential,
       'discrete': torch.distributions.Categorical,
       'uniform': torch.distributions.Uniform,
       'uniform-continuous': torch.distributions.Uniform,
       'flip': torch.distributions.Bernoulli,
       'vector': funcprimitives["vector"],
       'get': funcprimitives["get"],
       'put': funcprimitives["put"],
       'hash-map': funcprimitives["hash-map"],
       'first': funcprimitives["first"],
       'second': funcprimitives["second"],
       'last': funcprimitives["last"],
       'append': funcprimitives["append"],
       'conj': funcprimitives["conj"],
       'cons': funcprimitives["cons"],
       'list': funcprimitives["list"],
       '<': funcprimitives["less_than"],
       'mat-mul': torch.matmul,
       'mat-repmat': lambda x, y, z: x.repeat((int(y.item()), int(z.item()))),
       'mat-add': torch.add,
       'mat-tanh': torch.tanh,
       'mat-transpose': torch.t,
       'rest': funcprimitives["rest"],
       '=' : funcprimitives["="],
       '>': funcprimitives[">"],
       'empty?': funcprimitives["empty?"],
       'log': torch.log,
       'peek': funcprimitives['peek'],
       'push-address': funcprimitives['push_addr'],
       })
    return env

In [4]:
def evaluate_helper(x, env):
    "Evaluate an expression in an environment."
    if type(x) is str and x != 'fn':    # variable reference
        try:
            return env.find(x)[x]
        except AttributeError:
            return x
    
    elif type(x) in [int, float]: # constant 
        return torch.tensor(float(x))
    
    elif type(x) is torch.Tensor:
        return x
    
    op, *args = x 
    
    if op == 'if':             # conditional
        (test, conseq, alt) = args
        exp = (conseq if evaluate_helper(test, env) else alt)
        return evaluate_helper(exp, env)
            
    elif op == 'fn':         # procedure
        (parms, body) = args
        
        env_inner = Env(outer=env)
        return Procedure(parms[1:], body, env_inner)
    
    elif op == 'sample':
        d = evaluate_helper(args[1], env)
        return d.sample()
    
    elif op == 'observe':
        return evaluate_helper(args[-1], env)

    else:                        # procedure call
        proc = evaluate_helper(op, env) 
        push_address = env.find("push-address")["push-address"](*args[0][1:])  # push-address
        args_noaddres = args[1:]
        vals = [evaluate_helper(arg, env) for arg in args_noaddres]  
        return proc(*vals)

In [15]:
def evaluate(exp, env=None): #TODO: add sigma, or something

    if env is None:
        env = standard_env()
        
    return  evaluate_helper(exp, env)("")

def get_stream(exp):
    while True:
        yield evaluate(exp)

In [10]:
from daphne import daphne
from tests import is_tol, run_prob_test,load_truth
def run_deterministic_tests():
    
    for i in range(1,14):

        exp = daphne(['desugar-hoppl', '-i', '../HW5/programs/tests/deterministic/test_{}.daphne'.format(i)])
        truth = load_truth('programs/tests/deterministic/test_{}.truth'.format(i))
        ret = evaluate(exp)
        try:
            assert(is_tol(ret, truth))
        except:
            raise AssertionError('return value {} is not equal to truth {} for exp {}'.format(ret,truth,exp))
        
        print('FOPPL Tests passed')
        
    for i in range(1,13):

        exp = daphne(['desugar-hoppl', '-i', '../HW5/programs/tests/hoppl-deterministic/test_{}.daphne'.format(i)])
        truth = load_truth('programs/tests/hoppl-deterministic/test_{}.truth'.format(i))
        ret = evaluate(exp)
        try:
            assert(is_tol(ret, truth))
        except:
            raise AssertionError('return value {} is not equal to truth {} for exp {}'.format(ret,truth,exp))
        
        print('Test passed')
        
    print('All deterministic tests passed')
    


def run_probabilistic_tests():
    
    num_samples=1e4
    max_p_value = 1e-2
    
    for i in range(1,7):
        exp = daphne(['desugar-hoppl', '-i', '../HW5/programs/tests/probabilistic/test_{}.daphne'.format(i)])
        truth = load_truth('programs/tests/probabilistic/test_{}.truth'.format(i))
        
        stream = get_stream(exp)
        
        p_val = run_prob_test(stream, truth, num_samples)
        
        print('p value', p_val)
        assert(p_val > max_p_value)
    
    print('All probabilistic tests passed')    

In [13]:
run_deterministic_tests()

FOPPL Tests passed
FOPPL Tests passed
FOPPL Tests passed
FOPPL Tests passed
FOPPL Tests passed
FOPPL Tests passed
FOPPL Tests passed
FOPPL Tests passed
FOPPL Tests passed
FOPPL Tests passed
FOPPL Tests passed
FOPPL Tests passed
FOPPL Tests passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed
Test passed


AssertionError: return value tensor([4., 3., 2.]) is not equal to truth tensor([2, 3, 4]) for exp ['fn', ['alpha'], [['fn', ['alpha', 'map'], ['map', ['push-address', 'alpha', 'addr1'], ['fn', ['alpha', 'y'], ['+', ['push-address', 'alpha', 'addr2'], 1, 'y']], ['vector', ['push-address', 'alpha', 'addr3'], 1, 2, 3]]], ['push-address', 'alpha', 'addr0'], ['fn', ['alpha', 'f', 'values'], [['fn', ['alpha', 'f', 'values', 'map'], ['if', ['empty?', ['push-address', 'alpha', 'addr5'], 'values'], ['vector', ['push-address', 'alpha', 'addr6']], ['conj', ['push-address', 'alpha', 'addr7'], ['map', ['push-address', 'alpha', 'addr8'], 'f', ['rest', ['push-address', 'alpha', 'addr9'], 'values'], 'map'], ['f', ['push-address', 'alpha', 'addr10'], ['first', ['push-address', 'alpha', 'addr11'], 'values']]]]], ['push-address', 'alpha', 'addr4'], 'f', 'values', ['fn', ['alpha', 'f', 'values', 'map'], ['if', ['empty?', ['push-address', 'alpha', 'addr12'], 'values'], ['vector', ['push-address', 'alpha', 

In [16]:
run_probabilistic_tests()

('normal', 5, 1.4142136)
p value 0.6349776130087237
('beta', 2.0, 5.0)
p value 0.18276761492405472
('exponential', 0.0, 5.0)
p value 0.013595018538925328
('normal', 5.3, 3.2)
p value 0.9811860672852909
('normalmix', 0.1, -1, 0.3, 0.9, 1, 0.3)
p value 0.6543612355986826
('normal', 0, 1.44)
p value 0.9779379913777904
All probabilistic tests passed


In [17]:
for i in range(1,4):
    print(i)
    exp = daphne(['desugar-hoppl', '-i', '../HW5/programs/{}.daphne'.format(i)])
    print('\n\n\nSample of prior of program {}:'.format(i))
    print(evaluate(exp))        

1



Sample of prior of program 1:
tensor(54.)
2



Sample of prior of program 2:
tensor(1.7969)
3



Sample of prior of program 3:
tensor([0, 1, 2, 2, 1, 2, 2, 0, 1, 2, 1, 0, 2, 2, 1, 2, 2])
